#  GRID LSTM Model

**What is Meant by the GRID LSTM**
The GLSTM-A model is a proposed deep learning model designed for efficient short-term traffic flow prediction, addressing limitations in existing models like LSTM, TCN, and Conv-LSTM by focusing on temporal prediction without the need for spatial data. 

Key points of the model include:

**Parallel LSTM Grid with Attention:**

- The core architecture consists of a parallel LSTM grid network to extract periodicity information from long-term historical data, improving accuracy over traditional models.
- The attention mechanism enhances the model’s focus on important features, further boosting prediction accuracy.
Efficient Temporal Prediction:

- Unlike hybrid models (e.g., Conv-LSTM, TCN) that use spatial data, the GLSTM-A is tailored for scenarios where spatial information is not available.
It captures recurring patterns within specific time frames through LSTM layers, proficiently modeling temporal relationships in both long- and short-term data.

**Lower Memory Requirement:**

- The architecture is simpler than advanced models like Conv-LSTM and TCN, leading to reduced memory requirements, making it suitable for resource-constrained devices (e.g., Raspberry Pi).

GLSTM-A demonstrates its versatility in various traffic scenarios, making it suitable for intelligent transportation systems and resource-limited environments like edge devices.

The Below are the Required Modules for this Model

In [1]:
# Required Models for the project

import pandas as pd # type: ignore
import numpy as  np # type: ignore
import os # type: ignore
import math # type: ignore
import matplotlib.pyplot as plt # type: ignore

from sklearn.preprocessing import MinMaxScaler # type: ignore
from tensorflow.keras.models import Model , load_model, save_model # type: ignore
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Conv1D, TimeDistributed, Add, Multiply # type: ignore
from tensorflow.keras.losses import MeanSquaredError # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # type: ignore

Now, we have to load our dataset required for the Model

In [ ]:
file_path = 'PATH_TO_YOUR_DATASET.csv' # Path to your dataset
dataset = pd.read_csv(file_path) # Load the dataset

To make sure There is no zero values in the dataset,we have to check is there any null values in the dataset.

In [ ]:
null_values = dataset.isnull().sum() # Check for null values in the dataset
print("Null values in each column:\n", null_values) # Print the null values in the dataset

# Normalizing the Required Data in the dataset

Now, We have to get the required data values in between 0 and 1 to make the caculations simpler.

So, we are using the Normalizatioin Method.

In [ ]:
column = ['Column Name'] # Column name to be normalized
scaler = MinMaxScaler() # Normalizing the data
dataset[column] = scaler.fit_transform(dataset[column]) # Normalizing the data
dataset.head() # Displaying the first 5 rows of the dataset

# Sequence Creation

We have to divide the data into the sequences according to the required number of TimeIntervals.

In [ ]:
def create_lstm_sequences(data, time_steps, start_idx):
    X, Y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        Y.append(data[i + time_steps])
    return np.array(X), np.array(Y)

In [ ]:
time_steps = "YOUR_TIME_STEPS" # Time steps for the LSTM model
start_idx = 8064 # Start index for the dataset to take the 3 months data for the grid lstm
flow_data = dataset['Column Name'].values[start_idx:]

lstm_sequences = flow_data.reshape(-1, 1) # Reshape the data for the LSTM model
lstm_X, y = create_lstm_sequences(lstm_sequences, time_steps, start_idx) # Create LSTM sequences

# Print the shape of the LSTM input and output

print(lstm_X.shape)
print(y.shape)

Now, We have to reshape the data to the required shape so we are reshaping the data to the day wise format.

In [ ]:
def reshape_and_combine(df, files=13, rows_per_file=2016, rows_target=91):
    reshaped_data = []
    for i in range(files):
        start_idx = i * rows_per_file
        end_idx = start_idx + rows_per_file
        num_rows = end_idx - start_idx
        target_shape = (num_rows // 7, 7)
        reshaped_segment = df[start_idx:end_idx].reshape(target_shape)
        reshaped_data.append(reshaped_segment)
    final_data = np.concatenate(reshaped_data, axis=1)
    return pd.DataFrame(final_data)

reshaped_dataset = reshape_and_combine(dataset['Flow (Veh/5 Minutes)'].values)
reshaped_dataset = np.array(reshaped_dataset)

# Transpose the array to get the shape (91, 288)
reshaped_station = reshaped_dataset.T

print(reshaped_station.shape)  # Should output (91, 288)
print(reshaped_station)


In [ ]:
i = 0
j = 0
time_steps=4
k=time_steps-1
sequence_length=15
sequences=[]
sequence=[]

while i < reshaped_station.shape[0]-time_steps+1 and j <reshaped_station.shape[1]-sequence_length+1:
  sequence.append(reshaped_station[i][0+j:sequence_length+j])
  if i >= k:
    i=k-time_steps
    j += 1
    sequences.append(sequence)
    sequence=[]

  if  j >=  reshaped_station.shape[1]-sequence_length+1:
    k+=1
    j=0
    i=k-time_steps

  i += 1

print(np.array(sequences).shape)
print(np.array(sequences[0]).shape)

# Spliting the Data 

Now , we have to split the data for training and testing the data.

In [ ]:
min_sequences = min(len(lstm_X), len(sequences))

lstm_sequences = lstm_sequences[:min_sequences]
sequences = sequences[:min_sequences]
sequences = np.array(sequences)
split_idx = int(0.9 * min_sequences)

X_short_term_train = lstm_X[:split_idx]
X_short_term_test = lstm_X[split_idx:]
X_historical_train = sequences[:split_idx]
X_historical_test = sequences[split_idx:]
y_train = y[:split_idx]
y_test = y[split_idx:]

print(f"X_short_term_train shape: {X_short_term_train.shape}")
print(f"X_short_term_test shape: {X_short_term_test.shape}")
print(f"X_historical_train shape: {X_historical_train.shape}")
print(f"X_historical_test shape: {X_historical_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


# GRID LSTM Model Creation

We have to create the GRID LSTM Model as we already imported the required modeules respectively.

In [ ]:
def Create_Model():
    input0 = Input(shape=("shape of the conv layer"))
    input2 = Input(shape=("shape of the lstm layer"), name='input2')
    lstm_outputs=[]
    for i in range("range depends on the no of parellel lstm used in the model"):
        conv_layer = Conv1D(filters="mention no of filters needed", kernel_size=1, activation='', padding='')(input0)
        lstm_layer = LSTM("number of neurons needed", return_sequences=True, dropout=.2, name='lstm{}'.format(i))(conv_layer)
        lstm_layer1 = LSTM("number of neurins needed", return_sequences=True, dropout=.2)(lstm_layer)
        lstm_outputs.append(lstm_layer1)

    concatenated_output = Concatenate()(lstm_outputs)
    out = LSTM("number of neurins needed", activation='', return_sequences=False, name='out')(concatenated_output)
    dense = TimeDistributed(Dense("number of neurins needed", activation=''))(input2)
    lstm_out1 = LSTM("number of neurins needed", activation='relu', return_sequences=False, name='lstm_out1')(dense)
    x = Concatenate()([out, lstm_out1])
    output1 = Dense(1, activation='linear', name='output1')(x)
    model = Model(inputs=[input0, input2], outputs=output1)
    model.compile(optimizer='', loss='', metrics=[''])
    return model

model = Create_Model()

model.summary()

We created the model for the GRID LSTM, Now we have to train the model by using the Training Data.

In [ ]:
history = model.fit([X_historical_train, X_short_term_train], y_train, validation_split="Mention the validation split percentage" ,epochs="mention number of time you want to train the model" )

Upto to this is the model creation and the model training.

Let's Predict the values using the model we trained.

In [ ]:
predicted_values = model.predict([X_historical_test, X_short_term_test])

predictions_rescaled = scaler.inverse_transform(predicted_values.reshape(-1, 1)).flatten()
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

min_length = min(len(predictions_rescaled), len(y_test_rescaled))
predictions_rescale = predictions_rescaled[:min_length]
y_test_rescale = y_test_rescaled[:min_length]

results = pd.DataFrame(data={'Predictions': predictions_rescale, 'Actuals': y_test_rescale})

print(results)

# Saving the Trained Model

Now, we have to save the moedel so we can load the model without re-training the model.

In [ ]:
model.save("path_to_save_the_maodel") # Saving the model

In [ ]:
model1 = load_model("path_to_save_the_maodel") # Loading the model

# Loss Values of the Model 

Now , we have to check the loss values of the Model we trained so we can get the  accuracy of the model.

In [ ]:
rmse = math.sqrt(mean_squared_error(y_test_rescale, predictions_rescale))
mae = mean_absolute_error(y_test_rescale, predictions_rescale)
mape = mean_absolute_percentage_error(predictions_rescale, y_test_rescale)

# Displaying the RMSE, MAE and MAPE values

print('RMSE:', rmse)
print('MAE:', mae)
print('MAPE:', mape)

In [ ]:
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'Validation'], loc='upper left')
plt.show()